In [26]:
# read in the text, transforming everything to lower case
text_org = open('Beiras.txt').read().lower()
print('our original text has ' + str(len(text)) + ' characters')


our original text has 589161 characters


In [27]:
print(text_org[:1000])

 [http://resistiromorir.blogspot.com.es/p/xose-manuel-beiras-discurso-na.html]
 ben se decatarían xa as suas señorías de que o texto que veño de ler non está tirado de ningunha copla panfletaria contra as leoninas taxas impostas polo ministro de xustiza actual, malia que vulneran frontalmente o dereito dos cidadáns do común a unha xustiza gratuita, e reduce a cinzas a norma contida no artigo 24.1 da constitución, que predica que "tódalas persoas teñen dereito a obteren a tutela efectiva dos xuíces e tribunais no exercicio dos seus dereitos e intereses lexítimos, sen que, en ningún caso, se poida producir indefensión". porque resulta evidente que esa reaccionária decisión ministerial incurre na aberración de convirtir o recurso dos cidadáns á xustiza nunha mercaduría, un ben que compre mercar con cartos, so pena de ficar en indefensión.       non, señorías, non. ese texto non procede de ningún panfleto popular anónimo. tampouco é produto da malsán e rebirichada imaxinación do beiras. lo

In [28]:
import re
text_without_source="";
regexp=re.compile(r'http')
for line in text_org.splitlines():
    if not regexp.search(line):
        text_without_source= text_without_source + line
    

In [31]:
chars=sorted(list(set(text_without_source)))
print(chars);

[' ', '!', '"', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¡', 'ª', '«', '´', 'º', '»', '¿', 'à', 'á', 'â', 'ä', 'ç', 'è', 'é', 'ê', 'ì', 'í', 'ï', 'ñ', 'ó', 'ô', 'ö', 'ú', 'ü', '–', '—', '‘', '’', '“', '”', '•', '…']


In [36]:

text_clean = re.sub('[ºªàâäçèêïìôöü]|\W+',' ',text_without_source)
text_clean = text_clean.replace("  "," ")
chars=sorted(list(set(text_clean )))
print(chars);
print(len(text_clean))

[' ', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'á', 'é', 'í', 'ñ', 'ó', 'ú']
568575


In [35]:
print(text_clean[:1000])

 ben se decatarían xa as suas señorías de que o texto que veño de ler non está tirado de ningunha copla panfletaria contra as leoninas taxas impostas polo ministro de xustiza actual malia que vulneran frontalmente o dereito dos cidadáns do común a unha xustiza gratuita e reduce a cinzas a norma contida no artigo 24 1 da constitución que predica que tódalas persoas teñen dereito a obteren a tutela efectiva dos xuíces e tribunais no exercicio dos seus dereitos e intereses lexítimos sen que en ningún caso se poida producir indefensión porque resulta evidente que esa reaccionária decisión ministerial incurre na aberración de convirtir o recurso dos cidadáns á xustiza nunha mercaduría un ben que compre mercar con cartos so pena de ficar en indefensión non señorías non ese texto non procede de ningún panfleto popular anónimo tampouco é produto da malsán e rebirichada imaxinación do beiras lonxe diso ben se decatarían xa as suas cultas señorías de que é a reprodución literal dun punxente poem